In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame

In [13]:
import gym
import random

In [34]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [45]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    
    print("Episode:{} Score:{}".format(episode, score))

Episode:1 Score:14.0
Episode:2 Score:17.0
Episode:3 Score:29.0
Episode:4 Score:21.0
Episode:5 Score:14.0
Episode:6 Score:33.0
Episode:7 Score:14.0
Episode:8 Score:11.0
Episode:9 Score:17.0
Episode:10 Score:21.0


In [36]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [37]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [38]:
model = build_model(states, actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [39]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [40]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [41]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 28s 3ms/step - reward: 1.0000
102 episodes - episode_reward: 97.471 [9.000, 500.000] - loss: 2.445 - mae: 19.625 - mean_q: 39.746

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 28s 3ms/step - reward: 1.0000
54 episodes - episode_reward: 184.130 [134.000, 313.000] - loss: 3.747 - mae: 40.561 - mean_q: 81.693

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 1.0000
51 episodes - episode_reward: 194.784 [147.000, 405.000] - loss: 2.536 - mae: 42.196 - mean_q: 84.907

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 1.0000
48 episodes - episode_reward: 210.729 [142.000, 500.000] - loss: 1.545 - mae: 40.504 - mean_q: 81.636

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 30s 3ms/step - reward: 1.0000
done,

In [1]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

NameError: name 'dqn' is not defined